### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
#school_data_df.head(15)
len(student_data_df)#.head(15)
#school_data_complete_df.head()

39170

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# total schools
total_schools = school_data_complete_df["school_name"].nunique()

# total num students
total_students = len(school_data_complete_df)

# total budget
#group_by_school_df = school_data_complete_df.groupby(["school_name"])
total_budget = school_data_df["budget"].sum()

# average math score
ave_math = school_data_complete_df["math_score"].mean()

# average reading score
ave_reading = school_data_complete_df["reading_score"].mean()

# percent students with passing math score (70 or greater)
total_passing_math = len(school_data_complete_df.loc[school_data_complete_df["math_score"] >= 70, :])
percent_passing_math = total_passing_math / total_students * 100

# percent students with passing reading score (70 or greater)
total_passing_reading = len(school_data_complete_df.loc[school_data_complete_df["reading_score"] >= 70, :])
percent_passing_reading = total_passing_reading / total_students * 100
percent_passing_reading

# percent students with passing math AND reading score (70 or greater) - Calculate percent overall passing
total_passing_math_and_reading = len(school_data_complete_df.loc[ (school_data_complete_df["reading_score"] >= 70) &
                                                              (school_data_complete_df["math_score"] >= 70) ])
percent_passing_math_and_reading = total_passing_math_and_reading / total_students * 100
# percent_passing_math_and_reading
# percent_passing_reading
# percent_passing_math

# create a datagframe to hold all of the above data
summary_df = pd.DataFrame({"Total Schools" : [total_schools],
                          "Total Students" : f'{total_students:,}',
                          "Total Budget" : f'${total_budget:,}',
                          "Average Math Score" : f'%{ave_math:.2f}',
                          "Average Reading Score" : f'%{ave_reading:.2f}',
                          "% Passing Math" : f'%{percent_passing_math:.2f}',
                          "% Passing Reading" : f'%{percent_passing_reading:.2f}',
                          "% Overall Passing" : f'%{percent_passing_math_and_reading:.2f}'})
summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",%78.99,%81.88,%74.98,%85.81,%65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [28]:
# First grab the columns we need directly from the school dataframe
school_summary_df = school_data_df[["school_name", "type", "size", "budget"]]
school_summary_df = school_summary_df.rename(columns={"school_name": "School Name", "type": "School Type", "size": "Total Students", "budget": "Total School Budget"})
school_summary_df = school_summary_df.set_index("School Name")

# Need to divide the Budget column by the Students column for the Per Student Budget
school_summary_df['Per Student Budget'] = school_summary_df["Total School Budget"] / school_summary_df["Total Students"]

# Calculate average math and reading scores
# Students df --> Group by school --> average of math / reading scores
students_grouped_school = student_data_df.groupby('school_name')
means_df = students_grouped_school.mean()
# put the columns into the school summary df
school_summary_df['Average Math Score'] = means_df['math_score']
school_summary_df['Average Reading Score'] = means_df['reading_score']

school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score
School Name,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757


In [41]:
# Percent passing reading column
#bailey_pass_read = pr.loc["Bailey High School", "student_name"].count()/school_summary_df.loc["Bailey High School", "Total Students"]

# filter out passers from the student_data_df
passing_reading_students_df = student_data_df.loc[student_data_df['reading_score'] >= 70, :]
passing_reading_students_df = passing_reading_students_df.set_index("school_name")

# group by school
passing_reading_group = passing_reading_students_df.groupby('school_name')

# create a column for the % passing reading
school_summary_df['% Passing Reading'] = ""

# iterate over groups
for school, group in passing_reading_group:
    percent_pass = passing_reading_students_df.loc[school, "student_name"].count() / school_summary_df.loc[school, "Total Students"]
    school_summary_df.at[school, '% Passing Reading'] = percent_pass 


In [42]:
# Percent passing MATH column
# filter out passers from the student_data_df
passing_math_students_df = student_data_df.loc[student_data_df['math_score'] >= 70, :]
passing_math_students_df = passing_math_students_df.set_index("school_name")

# group by school
passing_math_group = passing_math_students_df.groupby('school_name')

# create a column for the % passing reading
school_summary_df['% Passing Math'] = ""

# iterate over groups
for school, group in passing_math_group:
    percent_pass = passing_math_students_df.loc[school, "student_name"].count() / school_summary_df.loc[school, "Total Students"]
    school_summary_df.at[school, '% Passing Math'] = percent_pass 
    
school_summary_df.head(16)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math
School Name,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.813164,0.656839
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.807392,0.659885
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,0.958546,0.938671
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.80863,0.66753
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.97139,0.933924
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.965396,0.938677
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.970398,0.941335
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.819333,0.666801
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.962529,0.925059


In [ ]:
# ADD COLUMN TO SCHOOL SUMMARY DF FOR % OVERALL PASSING STUDENTS


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type